In [1]:
import classify as c
import math
import os
import re

In [2]:
v=c.create_vocabulary('./EasyFiles/', 1)
v

[',',
 '.',
 '19',
 '2020',
 'a',
 'cat',
 'chases',
 'dog',
 'february',
 'hello',
 'is',
 'it',
 'world']

In [3]:
v2=c.create_vocabulary('./EasyFiles/', 2)
v2

['.', 'a']

In [4]:
# classify.create_vocabulary('./EasyFiles/2016/0.txt', 1)

In [5]:
def create_bow(vocab, filepath):
    """ Create a single dictionary for the data
        Note: label may be None
    """
    bow = {}
    with open(filepath) as doc:
        for word in doc:
            word = word.strip()
            if word not in vocab:
                if "None" not in bow: 
                    bow["None"]=1
                else:
                    bow["None"] +=1
            if not word in bow and word in vocab:
                bow[word] = 1
            elif word in vocab:
                bow[word] += 1
    return bow

In [6]:
create_bow(v2,'./EasyFiles/2016/1.txt')

{'a': 2, 'None': 3, '.': 1}

In [7]:
# >>> vocab = create_vocabulary('./EasyFiles/', 1)
# >>> load_training_data(vocab,'./EasyFiles/')
# => [{'label': '2020', 'bow': {'it': 1, 'is': 1, 'february': 1, '19': 1, ',': 1, '2020': 1, '.': 1}},
#     {'label': '2016', 'bow': {'hello': 1, 'world': 1}},
#     {'label': '2016', 'bow': {'a': 2, 'dog': 1, 'chases': 1, 'cat': 1, '.': 1}}]

In [8]:
vocab = c.create_vocabulary('./EasyFiles/', 1)
c.load_training_data(vocab,'./EasyFiles/')

[{'label': '2016', 'bow': {'hello': 1, 'world': 1}},
 {'label': '2016', 'bow': {'a': 2, 'dog': 1, 'chases': 1, 'cat': 1, '.': 1}},
 {'label': '2020',
  'bow': {'it': 1,
   'is': 1,
   'february': 1,
   '19': 1,
   ',': 1,
   '2020': 1,
   '.': 1}}]

In [9]:
vocab = c.create_vocabulary('./corpus/training/', 2)
vocab


['!',
 '#',
 '$',
 '%',
 '&',
 "'",
 "''",
 "'85",
 "'94",
 "'97",
 "'d",
 "'m",
 "'re",
 "'s",
 "'ve",
 '(1)',
 '(18)',
 '(2)',
 '(2016)',
 '(2017)',
 '(2019)',
 '(2020)',
 '(3)',
 '(38)',
 '(4)',
 '(40)',
 '(5)',
 '(8)',
 ',',
 '-',
 '--',
 '---------------',
 '------------------------------------------------------------',
 '-lcb-',
 '-lrb-',
 '-rcb-',
 '-rrb-',
 '.',
 '...',
 '.1',
 '0',
 '000',
 '1',
 '10',
 '100',
 '1000',
 '11',
 '12',
 '130',
 '135',
 '14',
 '15',
 '16',
 '17',
 '170',
 '18',
 '19',
 '1984',
 '1990s',
 '2',
 '20',
 '2000',
 '2001',
 '2006',
 '2007',
 '2013',
 '2014',
 '2015',
 '2015-2016',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2025',
 '2030',
 '2030s',
 '2035',
 '21st',
 '23',
 '24',
 '24\\/7',
 '25',
 '27',
 '28',
 '29',
 '3',
 '3.5',
 '30',
 '300',
 '35',
 '38',
 '39',
 '4',
 '40',
 '400',
 '41',
 '42',
 '43',
 '47',
 '5',
 '5,000',
 '50',
 '540',
 '6',
 '60',
 '60,000',
 '7',
 '7-8',
 '70',
 '71',
 '737-max',
 '75',
 '8',
 '9',
 '92m',
 '92s',
 '92t'

In [10]:
training_data = c.load_training_data(vocab,'./corpus/training/')
training_data

[{'label': '2016',
  'bow': {'i': 2,
   'would': 3,
   'like': 2,
   'to': 20,
   'use': 1,
   'one': 4,
   'of': 23,
   'my': 1,
   'free': 1,
   'None': 8,
   'days': 1,
   'for': 4,
   'this': 3,
   '.': 29,
   'artificial': 8,
   'intelligence': 8,
   'is': 13,
   'rapidly': 1,
   'developing': 1,
   'and': 14,
   'entering': 1,
   'different': 1,
   'aspects': 2,
   'community': 1,
   'life': 1,
   'transportation': 2,
   'the': 26,
   'fields': 2,
   'with': 4,
   'most': 1,
   'ai': 8,
   'technological': 3,
   'advancements': 1,
   'according': 3,
   'stanford': 2,
   'hundred': 3,
   'year': 4,
   'study': 7,
   'on': 5,
   'self-driving': 6,
   'cars': 5,
   'not': 6,
   'a': 7,
   'fiction': 1,
   'any': 3,
   'more': 3,
   'by': 5,
   '2020': 3,
   'those': 5,
   'will': 3,
   'be': 7,
   'widely': 2,
   'spread': 1,
   'very': 2,
   'briefly': 2,
   'mentions': 1,
   'current': 2,
   'large': 1,
   'issues': 2,
   'such': 5,
   'as': 4,
   'loss': 1,
   'attention': 3,
   

In [11]:
# >>> prior(training_data, ['2020', '2016'])
# => {'2020': -0.32171182103809226, '2016': -1.2906462863976689}

In [12]:
def prior(training_data, label_list):
    """ return the prior probability of the label in the training set
        => frequency of DOCUMENTS
    """
    smooth = 1 # smoothing factor
    logprob = {}
    count={}
    for i in label_list:
        count[i]=0
    for doc in training_data:
        if doc["label"] == label_list[0]:
            count[label_list[0]]+=1
        else:
            count[label_list[1]]+=1
    total=0
    for i in count:
        total+=count[i]
    for i in count:
        logprob[i]=math.log((count[i]+smooth)/(total+2))

    return logprob

c.prior(training_data, ['2020', '2016'])

{'2020': -0.3217118210380923, '2016': -1.2906462863976693}

In [13]:
vocab = c.create_vocabulary('./EasyFiles/', 1)
training_data = c.load_training_data(vocab, './EasyFiles/')
# >>> p_word_given_label(vocab, training_data, '2020')
# => {',': -2.3513752571634776, '.': -2.3513752571634776, '19': -2.3513752571634776, '2020': -2.3513752571634776, 'a': -3.044522437723423, 'cat': -3.044522437723423, 'chases': -3.044522437723423, 'dog': -3.044522437723423, 'february': -2.3513752571634776, 'hello': -3.044522437723423, 'is': -2.3513752571634776, 'it': -2.3513752571634776, 'world': -3.044522437723423, None: -3.044522437723423}
# >>> p_word_given_label(vocab, training_data, '2016')
# => {',': -3.091042453358316, '.': -2.3978952727983707, '19': -3.091042453358316, '2020': -3.091042453358316, 'a': -1.9924301646902063, 'cat': -2.3978952727983707, 'chases': -2.3978952727983707, 'dog': -2.3978952727983707, 'february': -3.091042453358316, 'hello': -2.3978952727983707, 'is': -3.091042453358316, 'it': -3.091042453358316, 'world': -2.3978952727983707, None: -3.091042453358316}

In [14]:
training_data

[{'label': '2016', 'bow': {'hello': 1, 'world': 1}},
 {'label': '2016', 'bow': {'a': 2, 'dog': 1, 'chases': 1, 'cat': 1, '.': 1}},
 {'label': '2020',
  'bow': {'it': 1,
   'is': 1,
   'february': 1,
   '19': 1,
   ',': 1,
   '2020': 1,
   '.': 1}}]

In [15]:
# def p_word_given_label(vocab, training_data, label):
#     """ return the class conditional probability of label over all words, with smoothing """
#     smooth = 1 # smoothing factor
#     word_prob = {}
#     for i in vocab: 
#         word_prob[i]=0
#     total=0
#     word_prob["None"]=0
#     for i in training_data:
#         if i["label"]== label:
#             for word in i["bow"]:
#                 total+=i["bow"][word]
#                 if word in vocab:
#                     word_prob[word] += i["bow"][word]
#                 else:
#                     word_prob["None"]+=i["bow"][word]
#     for i in word_prob:
#         word_prob[i]=math.log((word_prob[i]+smooth)/(total+len(vocab)+1))
#     return word_prob

c.p_word_given_label(vocab, training_data, '2016')

{',': -3.0910424533583156,
 '.': -2.3978952727983707,
 '19': -3.0910424533583156,
 '2020': -3.0910424533583156,
 'a': -1.9924301646902063,
 'cat': -2.3978952727983707,
 'chases': -2.3978952727983707,
 'dog': -2.3978952727983707,
 'february': -3.0910424533583156,
 'hello': -2.3978952727983707,
 'is': -3.0910424533583156,
 'it': -3.0910424533583156,
 'world': -2.3978952727983707,
 'None': -3.0910424533583156}

In [16]:
vocab = c.create_vocabulary('./EasyFiles/', 2)
training_data = c.load_training_data(vocab, './EasyFiles/')
c.p_word_given_label(vocab, training_data, '2020')
# => {'.': -1.6094379124341005, 'a': -2.302585092994046, None: -0.35667494393873267}
# >>> p_word_given_label(vocab, training_data, '2016')
# => {'.': -1.7047480922384253, 'a': -1.2992829841302609, None: -0.6061358035703157}

{'.': -1.6094379124341003,
 'a': -2.3025850929940455,
 'None': -0.35667494393873245}

In [17]:
c.p_word_given_label(vocab, training_data, '2016')

{'.': -1.7047480922384253,
 'a': -1.2992829841302609,
 'None': -0.6061358035703156}

In [18]:
training_data

[{'label': '2016', 'bow': {'None': 2}},
 {'label': '2016', 'bow': {'a': 2, 'None': 3, '.': 1}},
 {'label': '2020', 'bow': {'None': 6, '.': 1}}]

In [19]:
# def train(training_directory, cutoff):
#     """ return a dictionary formatted as follows:
#             {
#              'vocabulary': <the training set vocabulary>,
#              'log prior': <the output of prior()>,
#              'log p(w|y=2016)': <the output of p_word_given_label() for 2016>,
#              'log p(w|y=2020)': <the output of p_word_given_label() for 2020>
#             }
#     """
#     retval = {}
#     label_list = os.listdir(training_directory)
#     vocab=c.create_vocabulary(training_directory, cutoff)
#     training_data=c.load_training_data(vocab, training_directory)
#     retval['vocabulary']=vocab
#     retval['log prior']=c.prior(training_data,label_list)
#     retval['p(w|y=2016)']= p_word_given_label(vocab, training_data, '2016')
#     retval['p(w|y=2020)']= p_word_given_label(vocab, training_data, '2020')
#     return retval



# => {'vocabulary': ['.', 'a'],
#     'log prior': {'2020': -0.916290731874155, '2016': -0.5108256237659905},
#     'log p(w|y=2020)': {'.': -1.6094379124341005, 'a': -2.302585092994046, None: -0.35667494393873267},
#     'log p(w|y=2016)': {'.': -1.7047480922384253, 'a': -1.2992829841302609, None: -0.6061358035703157}}
c.train('./EasyFiles/', 2)


{'vocabulary': ['.', 'a'],
 'log prior': {'2016': -0.5108256237659907, '2020': -0.916290731874155},
 'p(w|y=2016)': {'.': -1.7047480922384253,
  'a': -1.2992829841302609,
  'None': -0.6061358035703156},
 'p(w|y=2020)': {'.': -1.6094379124341003,
  'a': -2.3025850929940455,
  'None': -0.35667494393873245}}

In [20]:
# >>> model = train('./corpus/training/', 2)
# >>> classify(model, './corpus/test/2016/0.txt')
# => {'vocabulary': ['.', 'a'],
#     'log prior': {'2020': -0.916290731874155, '2016': -0.5108256237659905},
#     'log p(w|y=2020)': {'.': -1.6094379124341005, 'a': -2.302585092994046, None: -0.35667494393873267},
#     'log p(w|y=2016)': {'.': -1.7047480922384253, 'a': -1.2992829841302609, None: -0.6061358035703157}}

In [21]:
model=c.train('./corpus/training/', 2)
# classify(model, './corpus/test/2016/0.txt')
# => {'log p(y=2020|x)': -3906.351945884105, 'log p(y=2016|x)': -3916.458747858926, 'predicted y': '2020'}


In [23]:
model

{'vocabulary': ['!',
  '#',
  '$',
  '%',
  '&',
  "'",
  "''",
  "'85",
  "'94",
  "'97",
  "'d",
  "'m",
  "'re",
  "'s",
  "'ve",
  '(1)',
  '(18)',
  '(2)',
  '(2016)',
  '(2017)',
  '(2019)',
  '(2020)',
  '(3)',
  '(38)',
  '(4)',
  '(40)',
  '(5)',
  '(8)',
  ',',
  '-',
  '--',
  '---------------',
  '------------------------------------------------------------',
  '-lcb-',
  '-lrb-',
  '-rcb-',
  '-rrb-',
  '.',
  '...',
  '.1',
  '0',
  '000',
  '1',
  '10',
  '100',
  '1000',
  '11',
  '12',
  '130',
  '135',
  '14',
  '15',
  '16',
  '17',
  '170',
  '18',
  '19',
  '1984',
  '1990s',
  '2',
  '20',
  '2000',
  '2001',
  '2006',
  '2007',
  '2013',
  '2014',
  '2015',
  '2015-2016',
  '2016',
  '2017',
  '2018',
  '2019',
  '2020',
  '2025',
  '2030',
  '2030s',
  '2035',
  '21st',
  '23',
  '24',
  '24\\/7',
  '25',
  '27',
  '28',
  '29',
  '3',
  '3.5',
  '30',
  '300',
  '35',
  '38',
  '39',
  '4',
  '40',
  '400',
  '41',
  '42',
  '43',
  '47',
  '5',
  '5,000',
  '5

In [22]:
# def classify(model, filepath):
#     retval = {}
#     year2016=model["log prior"]["2016"]
#     year2020=model["log prior"]["2020"]
#     bow = c.create_bow(model["vocabulary"], filepath)
#     for i in bow:
#         year2016+=model["p(w|y=2016)"][i]*bow[i]
#         year2020+=model["p(w|y=2020)"][i]*bow[i]
#     retval["log p(y=2016|x)"]=year2016
#     retval["log p(y=2020|x)"]=year2020
#     if max(year2016,year2020)==year2016:
#         retval['predicted y']="2016"
#     else:
#         retval['predicted y']="2020"


#     return retval

c.classify(model, './corpus/test/2016/0.txt')

{'log p(y=2016|x)': -3916.458747858928,
 'log p(y=2020|x)': -3906.351945884106,
 'predicted y': '2020'}